## Githun 연동하기

## 라이브러리 설치

In [32]:
# !pip install schedule

## 라이브러리 불러오기

In [106]:
# API 호출
# ## https://data.go.kr/tcs/dss/selectApiDataDetailView.do?publicDataPk=15057440
from urllib.request import Request, urlopen
# Request https://docs.python.org/ko/3/library/urllib.request.html#urllib.request.Request
# urlopen https://docs.python.org/ko/3/library/urllib.request.html#urllib.request.urlopen
from urllib.parse import urlencode, quote_plus
# urlenocde https://docs.python.org/ko/3/library/urllib.parse.html#urllib.parse.urlencode
# quote_plus https://docs.python.org/ko/3/library/urllib.parse.html#urllib.parse.quote_plus
from urllib.parse import unquote
import requests

# 시간 생성
import time
import datetime
from pytz import timezone, utc

# 폴더 자동 생성
import os

# 코드 스케줄링
import threading
import schedule

# XML 파싱
import xml.etree.ElementTree as ET

In [2]:
# 폴더 자동 생성 함수
# https://data-make.tistory.com/170
def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)

## KEY unquote

In [3]:
KEY = 'yEaR%2F3MDedRSlVJL%2F2pxnVg0yre1N5VF3RZ%2FUAt56MJ7J2mNpfqhUvy05pXV0uhHTVY7DbyCR8xmMaDdYga67Q%3D%3D' # 종현
# apiKEY = requests.utils.unquote(KEY)
apiKey = unquote(KEY)
# print(apiKey)
KEYSW = 'M%2B4%2FqUiadT8X8PhgFjaQLDu%2BIOgPMURfGsOX%2FmVxwHQVJgnVR%2FMPjDYXkuQNwUFbZXlfnX5Lls3SUCiCLIFjgQ%3D%3D'
apiKeySW = unquote(KEYSW)

## URL 생성

### ws.bus.go.kr url을 사용하는 경우 
1.   'ServiceKey' 대문자 S
2.   'busRouteId' 노선ID는 busRouteId

In [4]:
# getArrInfoByRouteAll
url = 'http://ws.bus.go.kr/api/rest/arrive/getArrInfoByRouteAll'
queryParams = '?' + urlencode({ quote_plus('ServiceKey') : apiKey, quote_plus('busRouteId') : '204000046' })

In [5]:
# getBusPosByRtid
# 노선ID로 차량들의 위치정보를 조회합니다
url = 'http://ws.bus.go.kr/api/rest/buspos/getBusPosByRtid'
queryParams = '?' + urlencode({ quote_plus('ServiceKey') : apiKey, quote_plus('busRouteId') : '204000046' })

In [6]:
url+queryParams

'http://ws.bus.go.kr/api/rest/buspos/getBusPosByRtid?ServiceKey=yEaR%2F3MDedRSlVJL%2F2pxnVg0yre1N5VF3RZ%2FUAt56MJ7J2mNpfqhUvy05pXV0uhHTVY7DbyCR8xmMaDdYga67Q%3D%3D&busRouteId=204000046'

### openapi url을 사용하는 경우 
1.   'serviceKey' 소문자 s
2.   'routeId' 노선ID는 routeId

In [7]:
# 노선번호목록조회 busrouteservice
# 해당 노선번호의 노선ID, 노선유형, 운행지역 등을 제공합니다.
url = 'http://openapi.gbis.go.kr/ws/rest/busrouteservice' 
queryParams = '?' + urlencode({ quote_plus('serviceKey') : apiKey, quote_plus('keyword') : '9000' })

In [8]:
# 노선정보항목조회 busrouteservice/info
# 해당 노선에 대한 노선번호, 기점/종점 정류소, 첫차/막차시간, 배차간격, 운행업체 등의 운행계획 정보를 제공합니다.
url = 'http://openapi.gbis.go.kr/ws/rest/busrouteservice/info'
queryParams = '?' + urlencode({ quote_plus('serviceKey') : apiKey, quote_plus('routeId') : '204000046' })

In [9]:
# 경유정류소목록조회 busrouteservice/info
# 해당 노선이 정차하는 경유정류소 목록과 정류소명, 중앙차로여부, 회차점, 좌표값 등을 제공합니다.
url = 'http://openapi.gbis.go.kr/ws/rest/busrouteservice/station'
queryParams = '?' + urlencode({ quote_plus('serviceKey') : apiKey, quote_plus('routeId') : '204000046' })

In [37]:
# 경유정류소목록조회 buslocationservice
# 해당 노선이 정차하는 경유정류소 목록과 정류소명, 중앙차로여부, 회차점, 좌표값 등을 제공합니다.
url = 'http://openapi.gbis.go.kr/ws/rest/buslocationservice'
queryParams = '?' + urlencode({ quote_plus('serviceKey') : apiKey, quote_plus('routeId') : '204000046' })

## @routeIdList 작성 선행

In [51]:
routeIdList = \
[
234000013,
234000130,
204000056,
204000046,
204000041,
204000070,
204000059,
204000060,
204000022,
204000039,
204000040,
204000067,
204000044,
204000147,
]

## @현재 시간 생성 함수

In [67]:
def getNowTime():
    fileDateTime = utc.localize(datetime.datetime.utcnow()).astimezone(timezone('Asia/Seoul'))       # 자정이 넘어간 시간 파일 경로가 바뀌는 문제를 해결해야 합니다.
    return fileDateTime

In [77]:
print(getNowTime())
print(type(getNowTime()))
print(getNowTime().isoformat()) # string
print(getNowTime().isoformat()[:10])
print(getNowTime().isoformat()[11:19])

2020-11-10 21:03:01.562470+09:00
<class 'datetime.datetime'>
2020-11-10T21:03:01.563471+09:00
2020-11-10
21:03:01


## @filePathList 생성

In [111]:
fileAPIName = 'buslocationservicePARSING'
filePathList = []

for routeId in routeIdList:
    

    # 코드를 실행하는 장치에 따라 달라지는 경로 입니다. 수정해서 사용하세요.
    fileDevicepath = 'C:/Users/jongh/OneDrive/School/석사/1-2/자료구조/프로젝트/5.DATA'
    # yourDevicepath = 'C:/Users'

    # 고정된 이름 및 변수에 의해 자동으로 생성 되는 이름 입니다. 변수를 확인하세요.
    fileRootPath = '/Master/dataAPI/'
    fileAPIName = fileAPIName
    fileRouteId = str(routeId)                                                                                   # 파일 이름에 routeId를 포함 시킬지 확인합니다.
    fileDateTime = getNowTime().isoformat()[:10]      # 자정이 넘어간 시간 파일 경로가 바뀌는 문제를 해결해야 합니다.
    fileDateStr = fileDateTime[2:10]
    fileExtension = '.txt'

    #파일의 폴더 경로와 이름을 각각 출력합니다.
    folderPath = str(fileDevicepath + fileRootPath + fileAPIName + '/' + fileRouteId)
    # print(folderPath)
    fileName = fileRouteId + '_' + fileDateStr + fileExtension
    # print(fileName)

    # 파일의 최종 경로를 출력합니다.
    filePath = str(folderPath + '/' + fileName)
    
    # 폴더를 생성합니다.
    createFolder(folderPath)
    
    # 파일 경로 리스트에 파일경로를 추가합니다.
    filePathList.append(filePath)
    
    print(filePath)

C:/Users/jongh/OneDrive/School/석사/1-2/자료구조/프로젝트/5.DATA/Master/dataAPI/buslocationservicePARSING/234000013/234000013_20-11-10.txt
C:/Users/jongh/OneDrive/School/석사/1-2/자료구조/프로젝트/5.DATA/Master/dataAPI/buslocationservicePARSING/234000130/234000130_20-11-10.txt
C:/Users/jongh/OneDrive/School/석사/1-2/자료구조/프로젝트/5.DATA/Master/dataAPI/buslocationservicePARSING/204000056/204000056_20-11-10.txt
C:/Users/jongh/OneDrive/School/석사/1-2/자료구조/프로젝트/5.DATA/Master/dataAPI/buslocationservicePARSING/204000046/204000046_20-11-10.txt
C:/Users/jongh/OneDrive/School/석사/1-2/자료구조/프로젝트/5.DATA/Master/dataAPI/buslocationservicePARSING/204000041/204000041_20-11-10.txt
C:/Users/jongh/OneDrive/School/석사/1-2/자료구조/프로젝트/5.DATA/Master/dataAPI/buslocationservicePARSING/204000070/204000070_20-11-10.txt
C:/Users/jongh/OneDrive/School/석사/1-2/자료구조/프로젝트/5.DATA/Master/dataAPI/buslocationservicePARSING/204000059/204000059_20-11-10.txt
C:/Users/jongh/OneDrive/School/석사/1-2/자료구조/프로젝트/5.DATA/Master/dataAPI/buslocationservicePARSING/2

## 폴더 생성하기

In [13]:
#createFolder(folderPath)

## API 호출

In [39]:
url = 'http://openapi.gbis.go.kr/ws/rest/buslocationservice'
for i in range(10):
    for routeId, filePath in zip(routeIdList, filePathList):
        textFile = open(filePath, 'a', encoding = 'utf-8', newline = '\n')
        # 파일 경로가 routeId에 맞게 바뀌는지 확인합니다. 
        # 파일을 덮어 쓰고 있지 않은지 확인합니다. 
        queryParams = '?' + urlencode({ quote_plus('serviceKey') : apiKey, quote_plus('routeId') : routeId })
        request = Request(url + queryParams)
        request.get_method = lambda: 'GET'
        oneLineXML = urlopen(request).read().decode('utf8')
        textFile.write(oneLineXML)
        textFile.write('\n')
    print(fileDateTime[11:19],'\t', i+1, '번째 반복문을 실행하였습니다.')
textFile.close()

20:08:57 	 1 번째 반복문을 실행하였습니다.
20:08:57 	 2 번째 반복문을 실행하였습니다.
20:08:57 	 3 번째 반복문을 실행하였습니다.
20:08:57 	 4 번째 반복문을 실행하였습니다.
20:08:57 	 5 번째 반복문을 실행하였습니다.
20:08:57 	 6 번째 반복문을 실행하였습니다.
20:08:57 	 7 번째 반복문을 실행하였습니다.
20:08:57 	 8 번째 반복문을 실행하였습니다.
20:08:57 	 9 번째 반복문을 실행하였습니다.
20:08:57 	 10 번째 반복문을 실행하였습니다.


## @Schedule 모듈 사용을 위한 API 호출 생성 함수 생성

In [100]:
def callAPIbuslocationservice(routeIdList, filePathList):
    pastTime = getNowTime()
    print(pastTime.isoformat(),'\t API [ buslocationservice ] 호출을 [ 시작 ] 하였습니다.')    
    for routeId, filePath in zip(routeIdList, filePathList):
        textFile = open(filePath, 'a', encoding = 'utf-8', newline = '\n')
        # 파일 경로가 routeId에 맞게 바뀌는지 확인합니다. 
        # 파일을 덮어 쓰고 있지 않은지 확인합니다. 
        queryParams = '?' + urlencode({ quote_plus('serviceKey') : apiKey, quote_plus('routeId') : routeId })
        request = Request(url + queryParams)
        request.get_method = lambda: 'GET'
        oneLineXML = urlopen(request).read().decode('utf8')
        textFile.write(oneLineXML)
        textFile.write('\n')
    currentTime = getNowTime()
    print(currentTime.isoformat(),'\t API [ buslocationservice ] 호출을 [ 완료 ] 하였습니다.\t 소요시간: ', currentTime - pastTime)    

In [101]:
# 함수 [ callAPIbuslocationservice ] 테스트
callAPIbuslocationservice(routeIdList, filePathList)

2020-11-10T21:14:39.881085+09:00 	 API [ buslocationservice ] 호출을 [ 시작 ] 하였습니다.
2020-11-10T21:14:41.275683+09:00 	 API [ buslocationservice ] 호출을 [ 완료 ] 하였습니다.	 소요시간:  0:00:01.394598


## Schedule 라이브러리를 사용하여 함수 호출 스케줄링

### schedule 테스트 코드

In [41]:
#특정 함수 정의
def printhello():
    print("Hello!")
 
schedule.clear()
schedule.every(1).minutes.do(printhello) #30분마다 실행
# schedule.every().monday.at("00:10").do(printhello) #월요일 00:10분에 실행
# schedule.every().day.at("10:30").do(job) #매일 10시30분에 
 
#실제 실행하게 하는 코드
while True:
    schedule.run_pending()
    time.sleep(1)

Hello!
Hello!


KeyboardInterrupt: 

## @schedule 적용된 함수 호출 - 함수 [ callAPIbuslocationservice ]

In [102]:
schedule.clear()
schedule.every(1).minutes.do(lambda: callAPIbuslocationservice(routeIdList, filePathList))

while True:
    schedule.run_pending()
    time.sleep(1)

2020-11-10T21:15:48.009400+09:00 	 API [ buslocationservice ] 호출을 [ 시작 ] 하였습니다.
2020-11-10T21:15:49.293882+09:00 	 API [ buslocationservice ] 호출을 [ 완료 ] 하였습니다.	 소요시간:  0:00:01.284482
2020-11-10T21:16:49.688359+09:00 	 API [ buslocationservice ] 호출을 [ 시작 ] 하였습니다.
2020-11-10T21:16:53.026846+09:00 	 API [ buslocationservice ] 호출을 [ 완료 ] 하였습니다.	 소요시간:  0:00:03.338487
2020-11-10T21:17:53.403925+09:00 	 API [ buslocationservice ] 호출을 [ 시작 ] 하였습니다.
2020-11-10T21:17:57.403548+09:00 	 API [ buslocationservice ] 호출을 [ 완료 ] 하였습니다.	 소요시간:  0:00:03.999623
2020-11-10T21:18:57.747194+09:00 	 API [ buslocationservice ] 호출을 [ 시작 ] 하였습니다.
2020-11-10T21:18:59.087649+09:00 	 API [ buslocationservice ] 호출을 [ 완료 ] 하였습니다.	 소요시간:  0:00:01.340455
2020-11-10T21:19:59.528077+09:00 	 API [ buslocationservice ] 호출을 [ 시작 ] 하였습니다.
2020-11-10T21:20:00.823270+09:00 	 API [ buslocationservice ] 호출을 [ 완료 ] 하였습니다.	 소요시간:  0:00:01.295193
2020-11-10T21:21:01.230273+09:00 	 API [ buslocationservice ] 호출을 [ 시작 ] 하였습니다.
2020-

KeyboardInterrupt: 

## @Schedule 모듈 사용을 위한 API 호출 생성 함수 생성

In [112]:
def parseAPIbuslocationservice(routeIdList, filePathList):
    pastTime = getNowTime()
    print(pastTime.isoformat(),'\t API [ buslocationservice ] 호출을 [ 시작 ] 하였습니다.')    
    for routeId, filePath in zip(routeIdList, filePathList):
        textFile = open(filePath, 'a', encoding = 'utf-8', newline = '\n')
        # 파일 경로가 routeId에 맞게 바뀌는지 확인합니다. 
        # 파일을 덮어 쓰고 있지 않은지 확인합니다. 
        queryParams = '?' + urlencode({ quote_plus('serviceKey') : apiKey, quote_plus('routeId') : routeId })
        request = Request(url + queryParams)
        request.get_method = lambda: 'GET'
        oneLineXML = urlopen(request).read().decode('utf8')
        
        xtree = ET.fromstring(oneLineXML)

        rows = ["endBus","lowPlate","plateNo","plateType","remainSeatCnt","routeId","stationId","stationSeq"]
    
        resultCode = int(xtree[1].find("resultCode").text)
        queryTime = xtree[1].find("queryTime").text
        textFile.write(str(routeId))
        textFile.write("\n")
        if resultCode == 0 :
            for node in xtree[2] :

                for index in rows :
                    n = node.find(index).text
                    textFile.write(n)
                    textFile.write(' ')
                textFile.write("\n")

        else :
            print("There is no bus\n")
        
    currentTime = getNowTime()
    print(currentTime.isoformat(),'\t API [ buslocationservice ] 호출을 [ 완료 ] 하였습니다.\t 소요시간: ', currentTime - pastTime)    

In [110]:
parseAPIbuslocationservice(routeIdList, filePathList)

2020-11-10T21:33:25.882365+09:00 	 API [ buslocationservice ] 호출을 [ 시작 ] 하였습니다.
2020-11-10T21:33:27.183718+09:00 	 API [ buslocationservice ] 호출을 [ 완료 ] 하였습니다.	 소요시간:  0:00:01.301353


In [ ]:
schedule.clear()
schedule.every(1).minutes.do(lambda: parseAPIbuslocationservice(routeIdList, filePathList))

while True:
    schedule.run_pending()
    time.sleep(1)

2020-11-10T21:35:27.425690+09:00 	 API [ buslocationservice ] 호출을 [ 시작 ] 하였습니다.
2020-11-10T21:35:28.728839+09:00 	 API [ buslocationservice ] 호출을 [ 완료 ] 하였습니다.	 소요시간:  0:00:01.303149
2020-11-10T21:36:29.276859+09:00 	 API [ buslocationservice ] 호출을 [ 시작 ] 하였습니다.
2020-11-10T21:36:30.734729+09:00 	 API [ buslocationservice ] 호출을 [ 완료 ] 하였습니다.	 소요시간:  0:00:01.457870
2020-11-10T21:37:31.196304+09:00 	 API [ buslocationservice ] 호출을 [ 시작 ] 하였습니다.
2020-11-10T21:37:32.512513+09:00 	 API [ buslocationservice ] 호출을 [ 완료 ] 하였습니다.	 소요시간:  0:00:01.316209
2020-11-10T21:38:32.958948+09:00 	 API [ buslocationservice ] 호출을 [ 시작 ] 하였습니다.
2020-11-10T21:38:34.280042+09:00 	 API [ buslocationservice ] 호출을 [ 완료 ] 하였습니다.	 소요시간:  0:00:01.321094
2020-11-10T21:39:34.753107+09:00 	 API [ buslocationservice ] 호출을 [ 시작 ] 하였습니다.
2020-11-10T21:39:36.577929+09:00 	 API [ buslocationservice ] 호출을 [ 완료 ] 하였습니다.	 소요시간:  0:00:01.824822
2020-11-10T21:40:37.081147+09:00 	 API [ buslocationservice ] 호출을 [ 시작 ] 하였습니다.
2020-